In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving newspam.csv to newspam (1).csv


In [ ]:
df = pd.read_csv(path_to_file , names = ['Category' , 'Message'] , header = None)

In [ ]:
df.head()

Category                                            Message
NaN  Category                                            Message
0.0       ham  Go until jurong point, crazy.. Available only ...
1.0       ham                      Ok lar... Joking wif u oni...
2.0      spam  Free entry in 2 a wkly comp to win FA Cup fina...
3.0       ham  U dun say so early hor... U c already then say...

In [ ]:
df

In [ ]:
# !pip install tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
Category       1      1                                            Message   
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
Category       
Category    1  
ham        30  
spam        4

In [ ]:
df['Category'].value_counts()

ham         4825
spam         747
Category       1
Name: Category, dtype: int64

In [ ]:
747/4825

0.15481865284974095

15% spam emails, 85% ham emails: This indicates class imbalance

In [ ]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [ ]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [ ]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 2)

In [ ]:
df_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [ ]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

Category                                            Message  spam
896.0       ham  Hope you are having a good week. Just checking in     0
3779.0      ham                            Then ur physics get a-?     0
3845.0      ham  Pls go there today  &lt;#&gt; . I dont want an...     0
2299.0     spam  Congrats! 1 year special cinema pass for 2 is ...     1
3673.0     spam  You have won a Nokia 7250i. This is what you g...     1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [ ]:
X_train.head(3)

1852.0    This is the 2nd time we have tried 2 contact u...
5051.0    Tick, tick, tick .... Where are you ? I could ...
4875.0    lyricalladie(21/F) is inviting you to be her f...
Name: Message, dtype: object

Now lets import BERT model and get embeding vectors for few sample statements

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

This code utilizes TensorFlow Hub to load two pre-trained BERT (Bidirectional Encoder Representations from Transformers) models and their preprocessing module. BERT is a powerful language representation model developed by Google, which is widely used for various natural language processing tasks.



(1).
 hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"): This line loads the pre-trained BERT preprocessing module from TensorFlow Hub. The URL https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3 points to a BERT model trained on English text (uncased version) with the version 3 of the model. The preprocessing module is responsible for tokenizing, padding, and encoding the input text in a format that can be fed into the BERT model.




(2).  hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"): This line loads the pre-trained BERT encoder module from TensorFlow Hub. The URL https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4 points to a BERT model with 12 layers (L), 768 hidden units (H), and 12 attention heads (A). This model is also trained on English text (uncased version) and is version 4 of the model. The encoder module is responsible for learning representations of the input text and capturing contextual information.



To summarize, this code loads two components of the BERT model:

(1). BERT Preprocessing Module: Responsible for tokenizing and preprocessing the input text to make it suitable for BERT model input.

(2). BERT Encoder Module: The actual BERT model, which learns to represent text in a meaningful way by capturing contextual information.

With these components loaded, you can use them in various NLP tasks such as text classification, sentiment analysis, question answering, etc., by feeding preprocessed text to the encoder and then applying a task-specific layer on top of the encoder's output for the specific task.

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435168 , -0.5132724 , -0.8884571 , ..., -0.7474884 ,
        -0.7531473 ,  0.91964483],
       [-0.87208354, -0.50543964, -0.94446665, ..., -0.85847497,
        -0.7174535 ,  0.88082975]], dtype=float32)>

This code defines a function get_sentence_embeding(sentences) that takes a list of sentences as input, processes them using the BERT model, and returns the embeddings (vectors representing the sentences) for each sentence.

Here's a step-by-step explanation of the code:

def get_sentence_embeding(sentences):: This line defines a function called get_sentence_embeding that takes a list of sentences as input. The function will return embeddings for these sentences using BERT.

preprocessed_text = bert_preprocess(sentences): This line preprocesses the input sentences using the BERT preprocessing module that was loaded earlier. The bert_preprocess function takes care of tokenizing, padding, and encoding the input sentences to prepare them for BERT input. The result of this line is the preprocessed text ready to be fed into the BERT model.

return bert_encoder(preprocessed_text)['pooled_output']: This line passes the preprocessed text to the BERT encoder module that was loaded earlier. The bert_encoder function computes the embeddings (vectors representing the sentences) for each sentence in the preprocessed text.




bert_encoder(preprocessed_text) applies the BERT model to the preprocessed text. The output is a dictionary containing various keys, and one of them is 'pooled_output'. 'pooled_output' represents the embedding of the entire sentence after processing through the BERT model. It captures the contextual information of the sentence and is commonly used for tasks like sentence-level classification.

return bert_encoder(preprocessed_text)['pooled_output'] returns the 'pooled_output' embeddings as the final output of the function.

Get embeding vectors for few sample words. Compare them using cosine similarity

In [ ]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [ ]:
cosine_similarity([e[1]],[e[2]])

array([[0.9850745]], dtype=float32)

In [ ]:
cosine_similarity([e[2]],[e[3]])

array([[0.84945345]], dtype=float32)

In [ ]:
cosine_similarity([e[3]],[e[4]])

array([[0.9872036]], dtype=float32)

cosine_similarity(v1, v2): This is a function that calculates the cosine similarity between two vectors v1 and v2. The cosine similarity formula is as follows:

cosine_similarity(v1, v2) = dot_product(v1, v2) / (norm(v1) * norm(v2))


where:

(1). dot_product(v1, v2) is the dot product of vectors v1 and v2.


(2). norm(v1) is the Euclidean norm (magnitude) of vector v1.


(3). norm(v2) is the Euclidean norm (magnitude) of vector v2.

Build Model
There are two types of models you can build in tensorflow.

(1) Sequential (2) Functional


In TensorFlow, there are two main ways to create models: the Sequential API and the Functional API. Both approaches allow you to build and train neural networks, but they have some differences in how you define and connect layers.

(1). Sequential API:


The Sequential API is a simple and straightforward way to create a linear stack of layers, where each layer has one input tensor and one output tensor. It is suitable for building feedforward neural networks or other networks with a single input and a single output. However, it may not be suitable for models with multiple inputs, multiple outputs, or complex architectures.

Example

In [ ]:
# import tensorflow as tf

# # Creating a sequential model
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Summary of the model
# model.summary()


(2).  Functional API:


The Functional API provides more flexibility and allows you to create more complex models with multiple inputs, multiple outputs, shared layers, or skip connections. It is particularly useful for building models with a directed acyclic graph (DAG) structure. The Functional API allows you to explicitly define the connections between layers, making it easier to create intricate architectures.

Example :

import tensorflow as tf

# Input layer
inputs = tf.keras.Input(shape=(input_dim,))

# Hidden layers
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(32, activation='relu')(x)

# Output layer
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

This code defines a neural network model that combines BERT (Bidirectional Encoder Representations from Transformers) layers with additional neural network layers for a specific task, such as text classification. It uses TensorFlow to create the model using the Functional API.

Here's a step-by-step explanation of the code:

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text'): This line creates an input layer called text_input, which will receive text data as input. The shape=() indicates that the input will be a single string (i.e., one text sample), and dtype=tf.string specifies that the input data type will be a string.

preprocessed_text = bert_preprocess(text_input): This line preprocesses the text data using the BERT preprocessing layer that you loaded earlier using bert_preprocess.

outputs = bert_encoder(preprocessed_text): This line feeds the preprocessed text data through the BERT encoder layer, which you loaded earlier using bert_encoder. The outputs will be a dictionary containing various keys, one of which is 'pooled_output'.

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output']): This line applies a dropout layer to the 'pooled_output' obtained from BERT. Dropout is a regularization technique that randomly sets a fraction of input units to zero during training to prevent overfitting.

l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l): This line adds a Dense (fully connected) layer with one output unit and a sigmoid activation function. The sigmoid activation function is commonly used for binary classification tasks, where the output value will be in the range of [0, 1], representing the probability of the positive class.

model = tf.keras.Model(inputs=[text_input], outputs=[l]): This line constructs the final model by specifying the input (text_input) and output (l) layers. The model takes a single text input and produces a single output representing the probability of the positive class for binary classification.

The final model combines the BERT layers for text encoding with additional layers for classification. This approach is common in transfer learning scenarios, where the pretrained BERT model is used as a feature extractor for text data and is then followed by task-specific layers to perform a specific NLP task such as sentiment analysis or text classification.

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [ ]:
len(X_train)

1120

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

Train the model

In [ ]:
model.fit(X_train , y_train , epochs = 4)  #TAKE 10

Epoch 1/10
35/35 [==============================] - 683s 19s/step - loss: 0.6228 - accuracy: 0.6679 - precision: 0.6703 - recall: 0.6607
Epoch 2/10
35/35 [==============================] - 636s 18s/step - loss: 0.5017 - accuracy: 0.8295 - precision: 0.8243 - recall: 0.8375
Epoch 3/10
35/35 [==============================] - 593s 17s/step - loss: 0.4470 - accuracy: 0.8429 - precision: 0.8189 - recall: 0.8804
Epoch 4/10
35/35 [==============================] - 595s 17s/step - loss: 0.4000 - accuracy: 0.8652 - precision: 0.8658 - recall: 0.8643
Epoch 5/10
35/35 [==============================] - 587s 17s/step - loss: 0.3910 - accuracy: 0.8482 - precision: 0.8218 - recall: 0.8893
Epoch 6/10
35/35 [==============================] - 585s 17s/step - loss: 0.3484 - accuracy: 0.8848 - precision: 0.8801 - recall: 0.8911
Epoch 7/10
35/35 [==============================] - 585s 17s/step - loss: 0.3311 - accuracy: 0.8848 - precision: 0.8659 - recall: 0.9107
Epoch 8/10
35/35 [=======================

In [ ]:
model.evaluate(X_test , y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

y_predicted.flatten() is used to convert a multi-dimensional array (in this case, a 2D array) into a 1D array. The flatten() function in numpy is used for this purpose.

flatten() returns a copy of the array, so the original y_predicted array remains unchanged unless you reassign it to the result of y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

Precision, Recall, F1-score, Support, Accuracy, Macro avg, and Weighted avg are common metrics used in binary and multiclass classification tasks to evaluate the performance of machine learning models. Let's explain each of them:

Precision:
Precision is a metric that measures the accuracy of positive predictions made by the model. It is calculated as the ratio of true positive (TP) predictions to the total number of positive predictions (true positives + false positives).

Recall (Sensitivity or True Positive Rate):
Recall is a metric that measures the ability of the model to identify all the positive instances correctly. It is calculated as the ratio of true positive (TP) predictions to the total number of actual positive instances (true positives + false negatives).

F1-score:
The F1-score is the harmonic mean of precision and recall. It provides a single score that balances both precision and recall. It is particularly useful when you want to consider both false positives and false negatives equally important.

Support:
Support represents the number of occurrences of each class in the true labels. It is useful in multiclass classification, where it gives an indication of how well the model performs for each class based on its representation in the data.

Accuracy:
Accuracy measures the overall correctness of the model's predictions. It is calculated as the ratio of the total number of correct predictions (true positives + true negatives) to the total number of instances in the dataset.

Macro avg:
Macro average calculates the average of the metrics (precision, recall, F1-score) for each class independently and then takes the unweighted mean of these values. It gives equal importance to each class, regardless of class imbalance.

Weighted avg:
Weighted average calculates the average of the metrics (precision, recall, F1-score) for each class independently and then takes the weighted mean of these values based on the class support. It gives more importance to the metrics of the larger classes.

In summary:

(1). Precision focuses on the accuracy of positive predictions.

(2). Recall focuses on the ability to identify positive instances correctly.

(3). F1-score balances precision and recall.

(4). Support shows the number of instances for each class in the true labels.

(5). Accuracy measures overall correctness.

(6). Macro avg treats each class equally in the average.

(7). Weighted avg considers class imbalance in the average.

Inference


In [ ]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

What is BERT ?

BERT, which stands for Bidirectional Encoder Representations from Transformers, is a powerful language representation model developed by Google. It has revolutionized the field of Natural Language Processing (NLP) by significantly improving the way machines understand and generate human language. Here's a comprehensive explanation of BERT:

Pre-training and Transfer Learning:
BERT is a pre-trained model, which means it has been trained on a massive amount of raw text from the internet. The training process involves predicting missing words in a sentence, where BERT is trained to understand the context of each word by considering the entire sentence bidirectionally. This pre-training process captures rich, contextualized representations of words and sentences, allowing BERT to learn intricate language patterns.

Bidirectional Context:
Unlike traditional language models (e.g., GPT-2), BERT reads the entire sentence bidirectionally. It uses a transformer-based architecture, where each word in the sentence is contextualized based on both the words that come before and after it. This bidirectional context enables BERT to capture deeper semantic relationships between words, making it highly effective in various NLP tasks.

Masked Language Model (MLM):
One of the pre-training tasks in BERT involves randomly masking (hiding) some words in a sentence and training the model to predict these masked words. This task forces BERT to learn representations that can understand the context of a word from its surrounding words, even if the word itself is missing.

Next Sentence Prediction (NSP):
Another pre-training task in BERT is Next Sentence Prediction, where the model is trained to predict if a pair of sentences appear consecutively in the original text. This task helps BERT understand the relationships between different sentences and is useful for tasks like question answering or sentiment analysis.

Fine-Tuning:
After pre-training, BERT can be fine-tuned on specific downstream NLP tasks. During fine-tuning, BERT is further trained on labeled data for tasks like text classification, named entity recognition, question answering, etc. Fine-tuning adapts the generic language understanding capabilities of BERT to the specifics of the task at hand.

Why Use BERT in NLP:

BERT excels at understanding the context of words, which is essential for NLP tasks.
It captures rich semantics and syntactic information from text data.
BERT's pre-training allows it to generalize well to various downstream tasks with minimal task-specific training data.
By fine-tuning BERT on specific tasks, we can achieve state-of-the-art performance on a wide range of NLP tasks with a single, powerful model.